In [3]:
"""
    Classificação de digitos escritos à mão com o uso da CNN
    Atividade do curso 'Deep Learning com Python de A a Z - O Curso Completo'
    Marcio Salmazo Ramos
"""

import matplotlib.pyplot as plt
from keras.datasets import mnist  # Importação da base de dados
from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten  # 3ª e 4ª etapa da CNN
from keras.layers import Conv2D, MaxPooling2D  # 1ª e 2ª etapa da CNN
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras import utils


<p>Realizando o tratamento de dados no Dataset</p>

In [4]:
'''
    Carregar a base de dados, já com a separação pré-definida para teste e treinamento
    x -> atributos previsores
    y -> classes da base
'''
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
sizeXTRAIN = xTrain.shape
sizeXTEST = xTest.shape
sizeYTRAIN = yTrain.shape
sizeYTEST = yTest.shape

print('Shape dos atributos previsores de teste', sizeXTEST)
print('Shape dos atributos previsores de treinamento', sizeXTRAIN)
print('Shape das classes de teste', sizeYTEST)
print('Shape das classes de treinamento', sizeYTRAIN)

'''
    Existem 60000 imagens para treinamento 28x28 px
    Existem 10000 imagens para teste 28x28 px
    Existem 10000 valores como resposta para as imagens de teste
    Existem 60000 valores como resposta para as imagens de treinamento
'''

Shape dos atributos previsores de teste (10000, 28, 28)
Shape dos atributos previsores de treinamento (60000, 28, 28)
Shape das classes de teste (10000,)
Shape das classes de treinamento (60000,)


'\n    Existem 60000 imagens para treinamento 28x28 px\n    Existem 10000 imagens para teste 28x28 px\n    Existem 10000 valores como resposta para as imagens de teste\n    Existem 60000 valores como resposta para as imagens de treinamento\n'

In [5]:
'''
    É necessário realizar uma transformação dos dados a fim de que o tensorflow
    consiga analizá-los. Neste caso, a dimensionalidade de cada imagem será diminuida,
    ou seja, apenas uma camada RGB será utilizada 
'''
trainForecasters = xTrain.reshape(xTrain.shape[0], 28, 28, 1)
testForecasters = xTest.reshape(xTest.shape[0], 28, 28, 1)

'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização futuramente
'''

trainForecasters = trainForecasters.astype('float32')
testForecasters = testForecasters.astype('float32')

In [6]:
'''
    Realizando a normalização (min/max normalization) a fim de que os valores dos pixels estejam
    entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

trainForecasters /= 255
testForecasters /= 255

In [7]:
'''
    Criação de dummy variables a fim de criar um encoder para 
    os diferentes valores.
    Uma variável Dummy é aquela que toma o valor de "zero" ou "um" 
    indicando a ausência ou presença de qualidades ou atributos. 
    Essas variáveis são usadas como dispositivos para classificar 
    dados em categorias mutuamente exclusivas.

    Esta etapa se faz necessária, uma vez que o problema exige 
    classificação em mais de duas classes

    Com método utils.to_categorical no keras, um array de valores
    inteiro que representem diferentes classes pode ser convertido em
    um array ou matriz de valores binários que representem os inteiros 
    no array original, exemplo:

    Arr = <1,2,3,4>
    Mat = utils.to_categorical(Arr, 4)

    Mat = <
        <0,0,0,1>
        <0,0,1,0>
        <0,0,1,1>
        <0,1,0,0>
    >
'''
trainClass = utils.to_categorical(yTrain, 10)
testClass = utils.to_categorical(yTest, 10)


<p>Aplicando a rede neural densa</p>

In [8]:
'''
    Aplicando o operador de convolução
    A variável 'classifier recebe o modelo sequencial, ou seja: 
    permite inserir camadas de uma rede neural em série, onde o output da primeira 
    camada serve como input da segunda, e assim por diante.'

    A primeira camada (referência à imagem) contém o resultado do operador de 
    convolução. Neste caso, é definido que serão utilizados 32 filtros ou
    detectores de características para cada convolução e o kernal utilizado
    será 3x3. Por fim, a função de ativação para cada convolução será o Relu

    A função batch_normalization realiza a normalização do valores 
    do mapa de características gerado. Dessa forma há uma otimização
    no processo
'''

classifier = Sequential()
classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [9]:
'''
    Criação de uma nova camada de convolução, seguindo os mesmos
    moldes das etapas anteriores (operador convolução, pooling e flattening)
'''

classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))


In [10]:
'''
    A terceira etapa se dá apenas pelo achatamento da matriz
    obtido após o processo de pooling na etapa anterior
    
    Obs: o flattening precisa ser aplicado apenas ao final de 
    todas as camadas de convolução
'''

classifier.add(Flatten())

In [11]:
'''
    A quarta etapa do processo se dá pela adição da rede neural densa
    tendo como entrada os valores provenientes do vetor obtido na 
    etapa anterior.

    Nesta implementação, a rede neural conta com duas camadas
    intermediárias contendo 128 neurônios, e para cada um de seus 
    neurônios a função de ativação será a função Relu. Além disso,
    também é definida a camada de saída (contendo 10 saídas) e tem 
    a função 'softmax' como função de ativação 

    A função Dropout serve para zerar uma determinada quantidade de 
    entradas a fim de otimizar o sistema e reduzir o overfitting, 
    uma vez que neste caso a quantidade de inputs é expressiva.
    Neste caso (0.2) 20% das entradas serão zeradas 
'''

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 10, activation = 'softmax'))

In [12]:
'''
    A rede criada será compilada. Para a função de custo é utilizada a categorical_crossentropy
    (também conhecida como 'softmax loss'), ela é utilizada quando a saída é composta por 
    múltiplas classes (neste caso temos as possibilidades de 1 a 10)

    O otimizador 'adam' trata-se de um método estocástico de descida gradiente baseado na estimativa 
    adaptativa de momentos de primeira e segunda ordem

    O parâmetro metrics calcula com que frequência as previsões são iguais aos rótulos. 
    Essa métrica cria duas variáveis locais, 'total' e 'count' para calcular a frequência com que 
    a previsão corresponde ao gabarito.
'''
classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

'''
    O método fit no Keras é responsável por efetuar o treinamento do modelo, nele serão passados como
    parâmetros:

    * Os dados previsores e suas respectivas classes para o treinamento
    * O 'batch_size' que define o número de amostras que serão propagadas pela rede, por exemplo:
        Supondo que haja 1.050 amostras de treinamento e queira configurar um batch_size igual a 100. 
        O algoritmo pega as primeiras 100 amostras (da 1ª à 100ª) do conjunto de dados de treinamento e treina a rede. 
        Em seguida, ele pega as segundas 100 amostras (da 101ª à 200ª) e treina a rede novamente.
        OBS: o batch_size requer menos uso da memória
    
    * As épocas indicam o número total de iterações de todos os dados de treinamento em um ciclo 
        para treinar o modelo de aprendizado de máquina.
    * O validation_data traz os dados previsores e suas respectivas classes de teste (para comparação)
'''
classifier.fit(trainForecasters, trainClass, batch_size = 128, epochs = 5, validation_data = (testForecasters, testClass))



Epoch 1/5


469/469 [==============================] - 13s 23ms/step - loss: 0.2042 - accuracy: 0.9361 - val_loss: 0.1438 - val_accuracy: 0.9574
Epoch 2/5
469/469 [==============================] - 12s 25ms/step - loss: 0.0628 - accuracy: 0.9813 - val_loss: 0.0353 - val_accuracy: 0.9894
Epoch 3/5
469/469 [==============================] - 12s 25ms/step - loss: 0.0446 - accuracy: 0.9867 - val_loss: 0.0404 - val_accuracy: 0.9859
Epoch 4/5
469/469 [==============================] - 11s 23ms/step - loss: 0.0373 - accuracy: 0.9886 - val_loss: 0.0326 - val_accuracy: 0.9893
Epoch 5/5
469/469 [==============================] - 11s 23ms/step - loss: 0.0298 - accuracy: 0.9906 - val_loss: 0.0351 - val_accuracy: 0.9888
